In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']="1"

import matplotlib.pyplot as plt

from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import pandas as pd
from arch import RingGeMNet, GeMNet, L2Norm, GeM
import re

from fastprogress import master_bar, progress_bar
%config InlineBackend.figure_format ='retina'

In [2]:
# ./index -> points to index dir
COMP_DATA_DIR = Path('.')

df = pd.DataFrame({'Image' : sorted(get_image_files(COMP_DATA_DIR / 'index'))})

In [3]:
df.head()

,Image
0,index/00001b917fe62185.jpg
1,index/00002469b818f290.jpg
2,index/0000298d976221f3.jpg
3,index/00002e4e382333bf.jpg
4,index/000036329c35b65c.jpg


In [4]:
NUM_WORKERS=8
SIZE = 256
DO_FULL_SIZE = False 

class ImageListAbsPath(ImageList):
    def open(self, fn:PathOrStr)->Image:
        return open_image(fn.replace('./',''))
    
#tfms = get_transforms(do_flip=False)
tfms = (None, None)
if DO_FULL_SIZE:
    BS=1
    data = (ImageList.from_df(df,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.NO)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
    data.train_dl.dl.batch_sampler.drop_last = False
if not DO_FULL_SIZE:
    BS=64
    data = (ImageList.from_df(df,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.SQUISH, size=SIZE)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
    data.train_dl.dl.batch_sampler.drop_last = False

In [5]:
arch = models.resnet152 #models.densenet201
model_fname =  'resnet152_i200_l1000-256'
basename_suffix = 'cut-extractor-2scales6patches-gem3'
size_fname = 'full' if DO_FULL_SIZE else str(SIZE)

basename = f'{model_fname or arch.__name__}_{size_fname}_{basename_suffix}.pth'
print(basename)

resnet152_i200_l1000-256_256_cut-extractor-2scales6patches-gem3.pth


In [6]:
custom_head = nn.Sequential(
    #nn.AdaptiveMaxPool2d(1),
    Flatten(),
    L2Norm())

class Extractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.l2norm = L2Norm()
        self.pool   = GeM(3.) #nn.AdaptiveMaxPool2d(1)
    def forward(self, x):
        b,d,ny,nx = x.shape
        
        f0  = self.l2norm(self.pool(x)).view(b,1,d)
#        f0  = self.l2norm(self.pool(x)).view(b,d)
        
        #f1  = self.l2norm(self.pool(x[...,ny//2-ny//4:ny//2+ny//4,nx//2-nx//4:nx//2+nx//4])).view(b,1,d)

        #f1_x0y0 = self.l2norm(self.pool(x[...,:ny//2,:nx//2])).view(b,1,d)
        #f1_x0y1 = self.l2norm(self.pool(x[...,ny//2:,:nx//2])).view(b,1,d)
        #f1_x1y0 = self.l2norm(self.pool(x[...,:ny//2,nx//2:])).view(b,1,d)
        #f1_x1y1 = self.l2norm(self.pool(x[...,ny//2:,nx//2:])).view(b,1,d)
        
        #return torch.cat((f0,f1,f1_x0y0,f1_x0y1,f1_x1y0,f1_x1y1), dim=1)
        #return torch.cat((f0,f1), dim=1)
        
        return f0

In [7]:
arch

<function torchvision.models.resnet.resnet152(pretrained=False, **kwargs)>

In [8]:
archcut = nn.Sequential(*list(arch(pretrained=True).children())[:-2],Extractor())

In [9]:
def cut_fn(x): 
    print(x)
    return True
TRAIN_CLASSES=1000
if False:
    learn = cnn_learner(data, arch,pretrained=True, custom_head=Extractor(),
                       metrics=[accuracy], cut= -1,
                       loss_func=nn.CrossEntropyLoss())

learn = Learner(data, archcut)



In [10]:
if model_fname:
    learn = learn.load(model_fname, strict=False)
else:
    model_fname = arch.__name__


/home/antor/miniconda3/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Extractor. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [11]:
learn.summary()

Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 128, 128]       9,408      True      
______________________________________________________________________
BatchNorm2d          [64, 128, 128]       128        True      
______________________________________________________________________
ReLU                 [64, 128, 128]       0          False     
______________________________________________________________________
MaxPool2d            [64, 64, 64]         0          False     
______________________________________________________________________
Conv2d               [64, 64, 64]         4,096      True      
______________________________________________________________________
BatchNorm2d          [64, 64, 64]         128        True      
______________________________________________________________________
Conv2d               [64, 64, 64]         36,864     True      
______________________________________________________________

In [12]:
InferenceNet =  learn.model #Extractor(split_model_idx(learn.model, [-2])[0])

In [13]:
if False:
    data.batch_size = 2
    InferenceNet.eval()
    o = InferenceNet.forward(data.one_batch()[0].cuda())
    print(o.shape)

In [14]:
NUM_WORKERS=30

qdf = pd.DataFrame({'Image' : sorted(get_image_files(COMP_DATA_DIR / 'test'))})
qdf.head()

,Image
0,test/000088da12d664db.jpg
1,test/0001623c6d808702.jpg
2,test/0001bbb682d45002.jpg
3,test/0002362830cfe3a3.jpg
4,test/000270c9100de789.jpg


In [15]:
BS=1 if DO_FULL_SIZE else 64
qdata = (ImageList.from_df(qdf,path='', cols=['Image'])
        .split_none()
        .label_const()
        .transform(tfms, 
                   resize_method=ResizeMethod.NO if DO_FULL_SIZE else ResizeMethod.SQUISH, 
                   size=None if DO_FULL_SIZE else SIZE)
        .databunch(bs=BS, num_workers=NUM_WORKERS)
        .normalize(imagenet_stats)
       ) 
qdata.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(qdata.train_ds)
qdata.train_dl.dl.batch_sampler.drop_last = False

In [16]:
basename

'resnet152_i200_l1000-256_256_cut-extractor-2scales6patches-gem3.pth'

In [ ]:
def extract_vectors_batched(data,model):
    model.cuda()
    model.eval()
    n_img = len(data.train_ds)
    bs = data.batch_size
    vectors = None

    with torch.no_grad():
        for idx, (img,label) in enumerate(progress_bar(data.train_dl)):
            st=idx*bs
            fin=min((idx+1)*bs, n_img)
            out = model(img).cpu()
            if vectors is None: vectors = torch.zeros(n_img, *out.shape[1:])
            vectors[st:fin,...] = out
    return vectors

def extract_vectors_batched_multi(data,model):
    model.cuda()
    model.eval()
    n_img = len(data.train_ds)
    bs = data.batch_size
    vectors = None
    hook = hook_outputs((learn.model[7], learn.model[6]))
    
    extractor = Extractor().cuda().eval()

    with torch.no_grad():
        for idx, (img,label) in enumerate(progress_bar(data.train_dl)):
            st=idx*bs
            fin=min((idx+1)*bs, n_img)
            model(img).cpu()
            if vectors is None:
                vectors = [torch.zeros(n_img, layer_output.shape[1]) for layer_output in hook.stored]
            for i, layer_output in enumerate(hook.stored):
                vectors[i][st:fin,...] = extractor(layer_output)
    return vectors

In [ ]:
if False:
    index_features = extract_vectors_batched_multi(data, InferenceNet)
    query_features = extract_vectors_batched_multi(qdata,InferenceNet)
    torch.save(query_features, f'query_{basename}')
    torch.save(index_features, f'index_{basename}')
else:
    query_features = torch.load( f'query_{basename}')
    index_features = torch.load( f'index_{basename}')

In [ ]:
#Now lets do the nearest neighbor search and create the submission
import faiss

query_fnames = [x.stem for x in qdf.Image.tolist()]
index_fnames = [x.stem for x in df.Image.tolist()]


In [ ]:
learn, InferenceNet, co, res, flat_config, cpu_index, index = None, None, None, None, None, None, None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def t_pcawhitenlearn(X):

    N = X.shape[0]

    # Learning PCA w/o annotations
    m = X.mean(dim=0, keepdim=True)
    Xc = X - m
    Xcov = Xc.t() @ Xc
    Xcov = (Xcov + Xcov.t()) / (2*N)
    eigval, eigvec = torch.symeig(Xcov,eigenvectors=True)
    order = eigval.argsort(descending=True)
    eigval = eigval[order]
    eigvec = eigvec[:, order]

    P = torch.inverse(torch.sqrt(torch.diag(eigval))) @ eigvec.t()
    
    return m, P

def t_whitenapply(X, m, P, dimensions=None):
    
    if not dimensions: dimensions = P.shape[1]

    X = (X-m) @ P[:,:dimensions]
    X = X / (torch.norm(X, dim=1, keepdim=True) + 1e-6)
    return X

def get_idxs_and_dists(_query_features, _index_features, index_type='', BS = 32):
    
    if False:
        index_transforms = []
        for index_transform in index_type.split(','):
            m = re.match(r'PCAW(\d+)?', index_transform)
            if m is not None:
                dimensions = int(m[1]) if m[1] is not None else _index_features.shape[-1]
                print(f"Applying {dimensions} PCA, Whitening and L2Norm...", end="")
                m, P = t_pcawhitenlearn(_index_features)
                _index_features = t_whitenapply(_index_features, m, P,dimensions=dimensions).unsqueeze(1)
                _query_features = t_whitenapply(_query_features, m, P,dimensions=dimensions).unsqueeze(1)
                print("done")

            elif index_transform not in ['L2norm']: index_transforms.append(index_transform)

        index_type = ','.join(index_transforms)
        print(index_type)
    else:
         _index_features = _index_features.unsqueeze(1)
         _query_features = _query_features.unsqueeze(1)
        
    if isinstance(_query_features, Tensor): query_features = _query_features.numpy()
    if isinstance(_index_features, Tensor): index_features = _index_features.numpy()
    max_hits = 300
    
    n_patches = query_features.shape[1]
    n_queries = query_features.shape[0]

    if False:
        query_features = query_features[:,::n_patches,:].squeeze(1).copy()
#        query_features = query_features.reshape(-1,query_features_shape[2])
        index_features = index_features.reshape(-1,index_features.shape[2])
    else:
        query_features = query_features[:,::n_patches,:].squeeze(1).copy()
        index_features = index_features[:,::n_patches,:].squeeze(1).copy()    
        n_patches = 1

    print(query_features.shape, index_features.shape, n_queries, n_patches)
    
    flat_config = faiss.GpuIndexFlatConfig()
    flat_config.device = 0
    res = faiss.StandardGpuResources()
    co = faiss.GpuMultipleClonerOptions()
    co.shard=True
    co.shard_type=1
    co.useFloat16=True
    _index = faiss.index_factory(index_features.shape[1], index_type)#, faiss.METRIC_INNER_PRODUCT)
    try:
        index = faiss.index_cpu_to_all_gpus(_index,co=co) #
        print("Index in GPU")
    except:
        index = _index
        print("Index in CPU")
    print("Training index...", end="")
    index.train(index_features)
    print("done")
    print("Adding features to index...", end="")
    index.add(index_features)
    print("done")
    out_dists = np.zeros((len(query_features), max_hits), dtype=np.float32)
    out_idxs  = np.zeros((len(query_features), max_hits), dtype=np.int32)
    NUM_QUERY = len (query_features)
    for ind in progress_bar(range(0, len(query_features), BS)):
        fin = ind+BS
        if fin > NUM_QUERY: fin = NUM_QUERY
        q_descs = query_features[ind:fin]
        D, I = index.search(q_descs, max_hits)
        out_dists[ind:fin] = D
        out_idxs[ind:fin] = I // n_patches
    return out_idxs, out_dists

In [ ]:
faiss.omp_get_max_threads()

In [ ]:
query_features[0].shape

In [ ]:
faiss.omp_set_num_threads(31)
index_type="PCAW256,L2norm,Flat"
#index_type="IVF4096,PQ16"
#index_type="Flat"

#out_idxs, out_dists = get_idxs_and_dists(
#    torch.cat((query_features[0],query_features[1]),dim=-1).squeeze(1), 
#    torch.cat((index_features[0],index_features[1]),dim=-1).squeeze(1), BS = 32*4, index_type=index_type)

out_idxs, out_dists = get_idxs_and_dists(
    query_features[0].squeeze(1), 
    index_features[0].squeeze(1), BS = 32*4, index_type=index_type)

In [ ]:
np.sort(out_dists.reshape((-1,int(out_idxs.shape[1]*1))), axis=1)

In [ ]:
sub_fname = 'test_submission.csv'
sample_df = pd.read_csv('retrieval_sample_submission.csv')
sample_df['images'] = ''

In [ ]:
i = 1
idx = out_idxs[i]
dst = out_dists[i]
u_idx = np.unique(idx,return_index=True)[1]
i_dst = dst[u_idx]
o_dst =np.argsort(i_dst)
print(idx, o_dst, i_dst, idx[u_idx[o_dst]])


In [ ]:
sub = {}
for i, query_fname in progress_bar(enumerate(query_fnames), total=len(query_fnames)):
    #_out_idxs=out_idxs.reshape(-1,200)[i][np.unique(out_idxs.reshape(-1,200)[i],return_index=True)[1]]
    idx = out_idxs[i]
    dst = out_dists[i]
    u_idx = np.unique(idx,return_index=True)[1]
    i_dst = dst[u_idx]
    o_dst =np.argsort(i_dst)
    _out_idxs = idx[u_idx[o_dst]]

    #_out_idxs=out_idxs[i][np.unique(out_idxs[i],return_index=True)[1]]
    ids = [index_fnames[x] for x in _out_idxs[:100]]
    sub[query_fname] = ' '.join(ids)

In [ ]:
sub_df = pd.DataFrame({'id' : list(sub.keys()), 'images':list(sub.values())})
sub_df = pd.concat([sub_df, sample_df]).drop_duplicates(subset=['id'])
sub_df.to_csv(sub_fname, index=False)

In [ ]:
sub_df.iloc[:4]

In [ ]:
def image_results(row, n= 12):
    r = [open_image(Path('test') / (row.id + '.jpg'))]
    r.extend([open_image(Path('index') / (id + '.jpg')) for id in row.images.split(' ')[:n]])
    return r
show_all(image_results(sub_df.iloc[4]),r=4,figsize=(20, 20))

In [ ]:
basename

In [ ]:
!kaggle competitions submit -c landmark-retrieval-2019 -f {sub_fname} -m '{basename} {index_type}'

In [ ]:
import time
time.sleep(10)

In [ ]:
!kaggle competitions submissions -c landmark-retrieval-2019 -v > submissions.csv

In [ ]:
submissions = pd.read_csv('submissions.csv')
submissions.iloc[0].publicScore